In [ ]:
import pandas as pd
import requests
import os
import time
from tqdm import tqdm
import numpy as np
# カレントディレクトリを.pyと合わせるために以下を実行
from pathlib import Path
if Path.cwd().name == "notebook":
    os.chdir("..")

In [ ]:
# CSVファイルからDataFrameを読み込む
df = pd.read_csv('data/input/user_info_cleansing.csv', dtype={'zipcode': str})

# 欠損値を「N」に置換
df['zipcode'] = df['zipcode'].fillna('N')

# APIキー（PositionStackで取得）
api_key = '89f36561dea6f8feb8bad2a40ed83480'

# 結果を保存するためのリスト
results = []

# バッチサイズ（一度に処理する郵便番号の数）
batch_size = 1

url = 'http://geoapi.heartrails.com/api/json?method=searchByPostal&postal='

In [ ]:
def batch_geocode(api_key, zipcode):
    """PositionStack APIを使用して郵便番号のリストをジオコードする"""
    response = requests.get(url + zipcode)
    data = response.json()
    if response.status_code != 200:
        print(f"エラー: {response.status_code, data, params}")
    # 'response' と 'location' キーの存在を確認
    if 'response' not in data or 'location' not in data['response']:
        return np.nan, np.nan
    # 空の時の処理
    if not data['response'] or not data['response']['location']:
        return np.nan, np.nan
    
#     print(data['response']['location'][0]['x'], data['response']['location'][0]['y'])
    return data['response']['location'][0]['x'], data['response']['location'][0]['y']

In [ ]:
# APIリクエストをバッチで実行
for i in tqdm(range(0, len(df))):
    # 'N'を含まない郵便番号を作成
    zipcode = [zipcode for zipcode in df['zipcode'][i:i + 1] if zipcode != 'N']
#     display(zipcode)
    if zipcode:
        zipcode = zipcode[0]
        # 緯度と経度のデータを取得
        latitude, longitude = batch_geocode(api_key, zipcode)

        # 新しいカラムをDataFrameに追加して値を設定
        df.loc[df['zipcode'] == zipcode, '経度'] = longitude
        df.loc[df['zipcode'] == zipcode, '緯度'] = latitude

    if i % 5 == 0:
       time.sleep(1) 
display(df)
df.to_csv('data/input/user_info_longitude_latitude.csv', index=True)